# 1. Design a NIN network

In [2]:
import numpy as np

# 思路
1. 确定输入的维度。
2. 确定卷积的形状。
3. 确定1*1的卷积。
4. 池化。
5. 输出。

In [ ]:
# 老师这个cell是在网上找的[参考](https://zh-v2.d2l.ai/chapter_convolutional-modern/nin.html)
import tensorflow as tf

def nin_block(num_channels, kernel_size, strides, padding):
    return tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(num_channels, kernel_size, strides=strides,
                               padding=padding, activation='relu'),
        tf.keras.layers.Conv2D(num_channels, kernel_size=1,
                               activation='relu'),
        tf.keras.layers.Conv2D(num_channels, kernel_size=1,
                               activation='relu')])

# 总结：
1. 一般卷积的操作有3个地方需要注意：
   1. batch的作用是对应输出的批次。
   2. filter是一组卷积核，这组卷积核的数量对应于输入的channel的数量。同时filter的数量决定了下一层的channel的数量。也就是说filter中的每一个卷积核会对应输入里面的每一个channel进行卷积，然后将filter中所有卷积核的结果再求和，得到输出channel中的一片。
   3. 卷积层之间变换的卷积核都是二维的。
2. 1*1的卷积操作需要注意的：
   1. 1*1的卷积不会改变输出卷积层相对于输入卷积层的长和宽（注意这里没有提到channel），但是一般都会修改channel的数量。
   2. 它的主要目的在于尽可能的提取非线性特征。
3. 卷积层向全连接层变化的时候的是采用了一种比较特殊的卷积操作。
   1. 每个filter就只有一个卷积核。
   2. 卷积核的形状和输入层的形状一致。这个形状包括长、宽、channel数。这个卷积核是一个**三维**的卷积核。和前面的卷积核不同，前面的卷积层向卷积层之间的卷积核都是**二维**的。
   3. filter的数量和全连接层向量的数量相同。
   4. 在tensorflow里面这一个操作是通过Flatten层来完成过渡的（注意这里只是过渡）。[参考中的1.4就做了说明](https://www.cnblogs.com/peng8098/p/keras_7.html)，官方手册里面并没有详细说明原理，只说明了结果。在实际的编写中尝试进行了使用这种卷积核的卷积操作，但是是无法在summary中看到降低维度的结果的。[已经下载下来参考也说得非常清晰](../references/whatisFullyConnectedLayer.pdf)。
4. 要注意使用池化层来降低模型的维度，不然模型的维度太高了PC电脑要运行超过1分钟。
5. 一定要注意输入输出的维度。这是在定制模型的过程中非常重要的地方。自己心里要设计好这种维度，要做到心中有数。
   1. [这个作者的使用tensorflow编写模型的方式非常值的借鉴](https://blog.csdn.net/qq_42308217/article/details/110209432)。在下面的cell里面也展示了。

In [13]:
# 这是我自己的实现。但是中间的卷积层向全连接层的过渡和我自己的理解不一样。
# 在tensorflow里面使用了flatten来完成卷积层向全连接层的过渡。
import numpy as np

# 使用tensorflow的实现。
import tensorflow as tf
from tensorflow.keras import layers
# input = np.arange(512*512*32, dtype=float).reshape(512, 512, 32)
# output = np.arange(128*128*64, dtype=float).reshape(128, 128, 64)

# 输出的通道数
input_filters = 1028

outputFilters =64
# input_dim=115, use_bias=True,
model = tf.keras.models.Sequential()
# model.add(layers.InputLayer(input_shape=(512, 512, 32), name="input", dtype="float32"))

# 如果没有理解错的画，Conv2D会自动适应上一层的通道数。也就是说每一个filter中卷积核的个数是输入层
# 的通道数。这也就解释了为什么没有使用3维卷积来处理类似(512, 512, 32)的情况。而如果使用3维的卷积
# 核，在适应输入的通道数这个维度，然后在使用model.build(input_shape=(1, 512, 512, 32))实现输入
# 的时候第一个维度表示的batch。也就是输入的批次数。如果只有1个批次，那么可以填1即可。
# 在使用model.add(layers.InputLayer(input_shape=(512, 512, 32), name="input", dtype="float32"))
# 实现的时候，就不用输入batch这个参数。
model.add(layers.Conv2D(128, kernel_size=5,
                        strides=2, activation="relu", padding="same", name="conv2D"))
# 最后的全连接层并不是通过dense来实现的，而是通过1*1的卷积来实现的。这里有点没有想明白。
# 难道不是通过一次1*1的卷积之后，后面的才是2层全连接层吗？
# model.add(layers.Dense(2048, activation="relu"))
# model.add(layers.Dense(2048, activation="relu"))
model.add(layers.MaxPool2D(pool_size=3, strides=2, name="firstPoolLayer"))
model.add(layers.Conv2D(256, kernel_size=1,
                        strides=1, activation="relu", padding="same", name="Conv2D_1m1_0"))
model.add(layers.MaxPool2D(pool_size=3, strides=2, name="secondPoolLayer"))
model.add(layers.Conv2D(512, kernel_size=1,
                        strides=1, activation="relu", padding="same", name="Conv2D_1m1_1"))
model.add(layers.MaxPool2D(pool_size=5, strides=3, name="LastPoolLayer"))
# 我理解的卷积层向全连接层的过渡是使用了一个和卷积层的长、宽、通道数一样的卷积核的卷积层，通道数是输出向量的长度
# 下面这一层就是。
# 是不是在使用dense的时候，tensorflow自适应的卷积核的大小？但是在输出summary的时候维度并没有变化。
# model.add(layers.Conv2D(128, kernel_size=31,
#                         strides=1, activation="relu", padding="same", name="Conv3DtoFullyConnected_1"))

# tensorflow好像使用的是flatten来实现卷积层向全连接层的变化的。
model.add(layers.Flatten(name="ConvolutiontoFullyConnected"))
model.add(layers.Dense(1028, activation="relu", name="FirstFullyConnectedLayer"))
model.add(layers.Dense(1028, activation="relu", name="SecondFullyConnectedLayer"))
# 最后为1个元素的输出作为
# model.add(layers.Dense(1, activation="relu", name="output"))
# 最后一个输出的形状是1*1*1028。这个地方还需要向老师确认。
model.add(layers.Reshape((1, 1, 1028), name="output"))

# input_shape 这里是指定输入的结构，其中第一个是batch，第二个和第三个是图片的长宽，
# 第三个是通道数，如果是RGB那么就是3。
model.build(input_shape=(1, 512, 512, 32))
model.summary()


Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2D (Conv2D)              (1, 256, 256, 128)        102528    
_________________________________________________________________
firstPoolLayer (MaxPooling2D (1, 127, 127, 128)        0         
_________________________________________________________________
Conv2D_1m1_0 (Conv2D)        (1, 127, 127, 256)        33024     
_________________________________________________________________
secondPoolLayer (MaxPooling2 (1, 63, 63, 256)          0         
_________________________________________________________________
Conv2D_1m1_1 (Conv2D)        (1, 63, 63, 512)          131584    
_________________________________________________________________
LastPoolLayer (MaxPooling2D) (1, 20, 20, 512)          0         
_________________________________________________________________
ConvolutiontoFullyConnected  (1, 204800)             

In [35]:
# 参考AlexNet实现。[参考](https://blog.csdn.net/qq_42308217/article/details/110209432)

from tensorflow.keras import layers, models, Model, Sequential
im_height=224
im_width=224
class_num=1000
input_image = layers.Input(shape=(im_height, im_width, 3), dtype="float32")  # output(None, 224, 224, 3)
x = layers.ZeroPadding2D(((1, 2), (1, 2)))(input_image)   #valid和same都不能满足输出，因此需要手动padding处理 output(None, 227, 227, 3)
x = layers.Conv2D(48, kernel_size=11, strides=4, activation="relu")(x)       # output(None, 55, 55, 48)
x = layers.MaxPool2D(pool_size=3, strides=2)(x) #padding默认等于valid  # output(None, 27, 27, 48)
x = layers.Conv2D(128, kernel_size=5, padding="same", activation="relu")(x)  # output(None, 27, 27, 128)
x = layers.MaxPool2D(pool_size=3, strides=2)(x)                              # output(None, 13, 13, 128)
x = layers.Conv2D(192, kernel_size=3, padding="same", activation="relu")(x)  # output(None, 13, 13, 192)
x = layers.Conv2D(192, kernel_size=3, padding="same", activation="relu")(x)  # output(None, 13, 13, 192)
x = layers.Conv2D(128, kernel_size=3, padding="same", activation="relu")(x)  # output(None, 13, 13, 128)
x = layers.MaxPool2D(pool_size=3, strides=2)(x)                              # output(None, 6, 6, 128)

x = layers.Flatten()(x)                         # output(None, 6*6*128=4608)
x = layers.Dropout(0.2)(x)
x = layers.Dense(2048, activation="relu")(x)    # output(None, 2048)
x = layers.Dropout(0.2)(x)
x = layers.Dense(2048, activation="relu")(x)    # output(None, 2048)
x = layers.Dense(class_num)(x)                  # output(None, 5)

predict = layers.Softmax()(x)#将输出转化成为一个概率分布

model = models.Model(inputs=input_image, outputs=predict)
model.summary()
# predict = layers.Softmax()(x)#将输出转化成为一个概率分布

# model = models.Model(inputs=input_image, outputs=predict)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 227, 227, 3)       0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 55, 55, 48)        17472     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 27, 27, 48)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 27, 27, 128)       153728    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 13, 13, 192)       221376

In [4]:
# 梯度使用
import tensorflow as tf

x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x
dy_dx = g.gradient(y, x) 
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)
